In [ ]:
#Import de librerías necesarias para el funcionamiento del script
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
import os
import time
from datetime import datetime, date
import datetime
from dateutil.rrule import rrule, DAILY

ModuleNotFoundError: No module named 'textblob'

In [ ]:
!pip install tweepy

In [ ]:
#Tokens de acceso a la api de Twitter
access_token = '1323581970090262529-wKL943mY1MnY2jOkQNzEMD7rDfa2JS'
access_token_secret = 'mqwSKhXgvFLVkPa87rrDuOg0oD2oXnZf073VGjTYzM1gd'
consumer_key = 'Tdk1QZfRlWev2QoPRJwlxrK7A'
consumer_secret = 'e5mzC5rsEJQdPmbknbphldbbvYsoIeiuUYjBGmaK7lafSblhLn'

In [ ]:
#Tokens de acceso a la api de Twitter
access_token = '1323581970090262529-wKL943mY1MnY2jOkQNzEMD7rDfa2JS'
access_token_secret = 'mqwSKhXgvFLVkPa87rrDuOg0oD2oXnZf073VGjTYzM1gd'
consumer_key = 'Tdk1QZfRlWev2QoPRJwlxrK7A'
consumer_secret = 'e5mzC5rsEJQdPmbknbphldbbvYsoIeiuUYjBGmaK7lafSblhLn'

In [ ]:
#Autentificación en la api y creación de instancia Tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [ ]:
def scraptweets(search_words, dates, numTweets, db_tweets):
  program_start = time.time()

  for idx,fecha in enumerate(dates):
    start_run = time.time()

    tweets = tweepy.Cursor(api.search, q=search_words, lang="es", since=fecha[0], until = fecha[1], tweet_mode='extended').items(numTweets)

    tweet_list = [tweet for tweet in tweets]

    noTweets = 0

    for tweet in tweet_list:
      username = tweet.user.screen_name
      acctdesc = tweet.user.description
      location = tweet.user.location
      following = tweet.user.friends_count
      followers = tweet.user.followers_count
      totaltweets = tweet.user.statuses_count
      usercreatedts = tweet.user.created_at
      tweetcreatedts = tweet.created_at
      retweetcount = tweet.retweet_count
      hashtags = tweet.entities['hashtags']

      try:
        text = tweet.retweeted_status.full_text
      except AttributeError:  # Not a Retweet
        text = tweet.full_text

      ith_tweet = [username, acctdesc, location, following, followers, totaltweets, usercreatedts, tweetcreatedts, retweetcount, text, hashtags]

      db_tweets.loc[len(db_tweets)] = ith_tweet

      noTweets += 1

    end_run = time.time()
    duration_run = round((end_run-start_run)/60, 2)

    print('no. of tweets scraped for run {} is {}'.format(idx + 1, noTweets))
    print('time take for {} run to complete is {} mins'.format(idx+1, duration_run))

    if idx != len(dates)-1:
      time.sleep(920) #15 minute sleep time

  to_csv_timestamp = date.today().strftime('%Y%m%d')

  filename = to_csv_timestamp + '_output.csv'

  db_tweets.to_csv(filename,encoding='utf-8-sig')
  
  program_end = time.time()  
  print('Scraping has completed!')
  print('Total time taken to scrap is {} minutes.'.format(round(program_end - program_start)/60, 2))

In [ ]:
#Inicializar variables
search_words = '"puto moro" OR "puto gitano" OR "gitano de mierda" OR "moro de mierda" OR "puto simio" OR "negro de mierda" OR "puto negro" OR inmigrante OR panchito OR sudaca -filter:retweets'
numTweets = 2500
date_until = date.today()
date_since = date_until - datetime.timedelta(days=9)
db_tweets = pd.DataFrame(columns=['username', 'acctdesc', 'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text', 'hashtags'])
dates = []

#Cálculos auxiliares
aux_since = date_since + datetime.timedelta(days=1)
aux_until = date_until + datetime.timedelta(days=1)
dates_since = list(rrule(DAILY, dtstart=date_since, until=date_until))
dates_until = list(rrule(DAILY, dtstart=aux_since, until=aux_until))
for i in range(len(dates_since)):
  dates.append([dates_since[i].strftime("%Y-%m-%d"), dates_until[i].strftime("%Y-%m-%d")])

scraptweets(search_words, dates, numTweets, db_tweets)

no. of tweets scraped for run 1 is 631
time take for 1 run to complete is 0.18 mins
no. of tweets scraped for run 2 is 1338
time take for 2 run to complete is 0.43 mins
no. of tweets scraped for run 3 is 1088
time take for 3 run to complete is 0.34 mins
no. of tweets scraped for run 4 is 858
time take for 4 run to complete is 0.28 mins
no. of tweets scraped for run 5 is 871
time take for 5 run to complete is 0.28 mins
no. of tweets scraped for run 6 is 986
time take for 6 run to complete is 0.33 mins
no. of tweets scraped for run 7 is 1233
time take for 7 run to complete is 0.45 mins
no. of tweets scraped for run 8 is 1385
time take for 8 run to complete is 0.5 mins
no. of tweets scraped for run 9 is 1229
time take for 9 run to complete is 0.47 mins
no. of tweets scraped for run 10 is 915
time take for 10 run to complete is 0.37 mins
Scraping has completed!
Total time taken to scrap is 141.66666666666666 minutes.
